libraries 

In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Cleaning

In [2]:
data = pd.read_csv(r'C:\Users\Carol\Documents\IronHack\data.submissions\Projetos\Projeto final\Dataset\googleplaystore.csv')

In [3]:
data.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
App               10841 non-null object
Category          10841 non-null object
Rating            9367 non-null float64
Reviews           10841 non-null object
Size              10841 non-null object
Installs          10841 non-null object
Type              10840 non-null object
Price             10841 non-null object
Content Rating    10840 non-null object
Genres            10841 non-null object
Last Updated      10841 non-null object
Current Ver       10833 non-null object
Android Ver       10838 non-null object
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


O formato data do Last Updated não me passou confiança. 

In [5]:
data['Last Updated'].max()

'September 9, 2017'

O feeling funcionou, no head que dei acima deu pra ver que tem datas superiores a set de 2017. Logo, vou formatar a data e visualizar novamente.

In [6]:
new = data['Last Updated'].str.split(' ', expand=True) 

In [7]:
day = new[1].str.replace(',', "")

In [8]:
month = new[0]

In [9]:
month.replace('January', '01', inplace = True)
month.replace('February','02', inplace = True)
month.replace('March','03', inplace = True)
month.replace('April', '04', inplace = True)
month.replace('May','05', inplace = True)
month.replace('June','06', inplace = True)
month.replace('July', '07', inplace = True)
month.replace('August', '08', inplace = True)
month.replace('September','09', inplace = True)
month.replace('October', '10', inplace = True)
month.replace('November','11', inplace = True)
month.replace('December','12', inplace = True)


In [10]:
year = new[2]

In [11]:
data['Last Updated'] = day + '/' + month + '/' + year

In [12]:
data['Last Updated'] = pd.to_datetime(data['Last Updated'],infer_datetime_format=True)

Optei por transformar as colunas numericas tratadas como object, em int

In [13]:
data["Reviews"] = data["Reviews"].replace('3.0M', '3000000')

In [14]:
data["Reviews"] = data["Reviews"].astype('int')

A coluna Price e Installs tem um caracter especial que vai me atrapalhar na hora de fazer calculos.

In [15]:
data['Price'] = data['Price'].str.replace('$','')

In [16]:
data["Price"] = data["Price"].replace('Everyone', '0')

In [17]:
data["Price"] = data["Price"].astype('float')

In [18]:
data['Installs'] = data['Installs'].str.replace('+','')

In [19]:
data['Installs'] = data['Installs'].str.replace(',','')

In [20]:
data['Installs'] = data['Installs'].str.replace('.','')

In [21]:
data['Installs'] = data['Installs'].str.replace('Free','50000')

In [22]:
data["Installs"] = data["Installs"].astype('float')

In [23]:
data['Category'] = data['Category'].replace('1.9','FAMILY')

Pirmeira observação é separar a coluna Genres, pois há mais de uma informação. Então separei isso em duas colunas pra visualizar melhor na analise.

In [24]:
new = data['Genres'].str.split(';', expand=True)

In [25]:
data['Genre1'] = new[0]

In [26]:
data['Genre2'] = new[1]

In [27]:
data.drop('Genres', axis = 1, inplace = True)

In [28]:
data.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Last Updated,Current Ver,Android Ver,Genre1,Genre2
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,10000.0,Free,0.0,Everyone,2018-07-01,1.0.0,4.0.3 and up,Art & Design,None
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000.0,Free,0.0,Everyone,2018-01-15,2.0.0,4.0.3 and up,Art & Design,Pretend Play
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,5000000.0,Free,0.0,Everyone,2018-01-08,1.2.4,4.0.3 and up,Art & Design,None
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,50000000.0,Free,0.0,Teen,2018-08-06,Varies with device,4.2 and up,Art & Design,None
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,100000.0,Free,0.0,Everyone,2018-06-20,1.1,4.4 and up,Art & Design,Creativity


Hora de tratar os missing values

In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 14 columns):
App               10841 non-null object
Category          10841 non-null object
Rating            9367 non-null float64
Reviews           10841 non-null int32
Size              10841 non-null object
Installs          10841 non-null float64
Type              10840 non-null object
Price             10841 non-null float64
Content Rating    10840 non-null object
Last Updated      10840 non-null datetime64[ns]
Current Ver       10833 non-null object
Android Ver       10838 non-null object
Genre1            10841 non-null object
Genre2            498 non-null object
dtypes: datetime64[ns](1), float64(3), int32(1), object(9)
memory usage: 1.1+ MB


In [30]:
data.corr()

,Rating,Reviews,Installs,Price
Rating,1.000000,0.067645,0.048654,-0.021176
Reviews,0.067645,1.000000,0.643084,-0.009672
Installs,0.048654,0.643084,1.000000,-0.011688
Price,-0.021176,-0.009672,-0.011688,1.000000


# Missing Values, irei fazer a solução mais simples e mais efetiva. Nem todas serão perfeitas para os modelos mas acredito que no balanço final o impacto não será grande.

Tratarei os missing values (13%) em Rating como a média da variavel. Com o tempo escasso escolhi a alternativa mais simples pra resolver isso.

In [31]:
data['Rating'].fillna(data['Rating'].mean(), inplace = True)

Coluna Type tem apenas 1 missing, vou classifica lo como free apenas para não dropar a linha.

In [32]:
data['Type'].fillna('Free', inplace=True)

Content Rating, mais uma vez apenas 1 missing. Irei classificar como Everyone.

In [33]:
data['Content Rating'].fillna('Everyone', inplace=True)

Last Updated vou colocar a menor data do dataset e assumir que ele não teve updates.

In [34]:
data['Last Updated'].fillna(data['Last Updated'].min(), inplace=True)

Current Ver vou trata los com Varies with device, por que fala alguma coisa mesmo não explicando nada e não é 'mentira'.

In [35]:
data['Current Ver'].fillna('Varies with device', inplace=True)

Android Ver também tratarei com a categoria genérica Varies with device.

In [36]:
data['Android Ver'].fillna('Varies with device', inplace=True)

Genre 2 tem muitos missing por que nem todos os apps tem um sub genero. Então irei trata los como uma categoria a parte.

In [37]:
data['Genre2'].fillna('None', inplace=True)

## Exploratória!

In [38]:
data.shape

(10841, 14)

In [39]:
data['Last Updated'].min()

Timestamp('2010-05-21 00:00:00')

In [40]:
data['Last Updated'].max()

Timestamp('2018-12-07 00:00:00')

In [41]:
data.describe()

,Rating,Reviews,Installs,Price
count,10841.000000,1.084100e+04,1.084100e+04,10841.000000
mean,4.193338,4.443887e+05,1.546292e+07,1.027273
std,0.499557,2.927728e+06,8.502557e+07,15.948971
min,1.000000,0.000000e+00,0.000000e+00,0.000000
25%,4.100000,3.800000e+01,1.000000e+03,0.000000
50%,4.200000,2.094000e+03,1.000000e+05,0.000000
75%,4.500000,5.479800e+04,5.000000e+06,0.000000
max,19.000000,7.815831e+07,1.000000e+09,400.000000


In [42]:
data['Category'].value_counts()

FAMILY                 1973
GAME                   1144
TOOLS                   843
MEDICAL                 463
BUSINESS                460
PRODUCTIVITY            424
PERSONALIZATION         392
COMMUNICATION           387
SPORTS                  384
LIFESTYLE               382
FINANCE                 366
HEALTH_AND_FITNESS      341
PHOTOGRAPHY             335
SOCIAL                  295
NEWS_AND_MAGAZINES      283
SHOPPING                260
TRAVEL_AND_LOCAL        258
DATING                  234
BOOKS_AND_REFERENCE     231
VIDEO_PLAYERS           175
EDUCATION               156
ENTERTAINMENT           149
MAPS_AND_NAVIGATION     137
FOOD_AND_DRINK          127
HOUSE_AND_HOME           88
LIBRARIES_AND_DEMO       85
AUTO_AND_VEHICLES        85
WEATHER                  82
ART_AND_DESIGN           65
EVENTS                   64
COMICS                   60
PARENTING                60
BEAUTY                   53
Name: Category, dtype: int64

In [43]:
data = pd.get_dummies(data, prefix='Category')

In [ ]:
sns.pairplot(data)

In [ ]:
sns.distplot(data['Rating'])

In [ ]:
data.corr()

In [ ]:
plt.figure(figsize=(10,5))

sns.heatmap(data.corr(), annot=True)

In [ ]:
data.columns

# Regressão Linear

In [ ]:
y = data['Rating']
X = data.drop('Rating', axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lm = LinearRegression()

In [ ]:
lm.fit(X_train,y_train)

In [ ]:
'Égalité et Réconciliation' in data